<a href="https://colab.research.google.com/github/shallynagfase9/Naive-Bayes-Ensemble-Techniques-its-types/blob/main/Boosting_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Q1. What is Gradient Boosting Regression?

In [ ]:
"""
Gradient Boosting Regression is a machine learning technique used for regression tasks that involves building an ensemble of decision trees sequentially.
It belongs to the family of boosting algorithms and is particularly effective in creating predictive models where the goal is to predict a continuous numeric value (regression).

"""

#Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a simple regression problem as an example and train the model on a small dataset. Evaluate the model's performance using metrics such as mean squared error and R-squared.

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

class DecisionTreeRegressor:
    def __init__(self, max_depth=1):
        self.max_depth = max_depth
        self.tree = {}

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        if depth >= self.max_depth:
            return np.mean(y)  # leaf node value is mean of y

        feature_index, threshold, gain = self._find_best_split(X, y)

        if gain == 0:
            return np.mean(y)  # if no gain, return mean of y for leaf node

        left_indices = X[:, feature_index] < threshold
        right_indices = ~left_indices

        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return {'feature_index': feature_index,
                'threshold': threshold,
                'left_subtree': left_subtree,
                'right_subtree': right_subtree}

    def _find_best_split(self, X, y):
        best_feature_index = None
        best_threshold = None
        best_gain = 0

        m, n = X.shape
        base_error = np.var(y)

        for feature_index in range(n):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_indices = X[:, feature_index] < threshold
                right_indices = ~left_indices

                if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                    continue

                left_variance = np.var(y[left_indices])
                right_variance = np.var(y[right_indices])

                weighted_variance = (np.sum(left_indices) / m) * left_variance + \
                                    (np.sum(right_indices) / m) * right_variance

                gain = base_error - weighted_variance

                if gain > best_gain:
                    best_gain = gain
                    best_feature_index = feature_index
                    best_threshold = threshold

        return best_feature_index, best_threshold, best_gain

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, tree):
        if isinstance(tree, dict):
            feature_index = tree['feature_index']
            threshold = tree['threshold']

            if x[feature_index] < threshold:
                return self._predict_tree(x, tree['left_subtree'])
            else:
                return self._predict_tree(x, tree['right_subtree'])
        else:
            return tree

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        y_pred = np.full_like(y, np.mean(y))  # initialize predictions with mean of y

        for _ in range(self.n_estimators):
            residuals = y - y_pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.estimators.append(tree)
            y_pred += self.learning_rate * tree.predict(X)

    def predict(self, X):
        y_pred = np.zeros(len(X))
        for estimator in self.estimators:
            y_pred += self.learning_rate * estimator.predict(X)
        return y_pred

def evaluate_performance(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, r2

# Example usage with a simple regression dataset
np.random.seed(42)
X = np.random.rand(100, 1) * 10
y = 2 * X[:, 0] + np.random.randn(100)  # linear relationship with noise

gb_regressor = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=2)
gb_regressor.fit(X, y)

y_pred = gb_regressor.predict(X)

mse, r2 = evaluate_performance(y, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")


Mean Squared Error: 88.5412
R-squared: -1.5844


#Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth to optimise the performance of the model. Use grid search or random search to find the best hyperparameters

In [3]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

class DecisionTreeRegressor:
    def __init__(self, max_depth=1):
        self.max_depth = max_depth
        self.tree = {}

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        if depth >= self.max_depth:
            return np.mean(y)  # leaf node value is mean of y

        feature_index, threshold, gain = self._find_best_split(X, y)

        if gain == 0:
            return np.mean(y)  # if no gain, return mean of y for leaf node

        left_indices = X[:, feature_index] < threshold
        right_indices = ~left_indices

        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return {'feature_index': feature_index,
                'threshold': threshold,
                'left_subtree': left_subtree,
                'right_subtree': right_subtree}

    def _find_best_split(self, X, y):
        best_feature_index = None
        best_threshold = None
        best_gain = 0

        m, n = X.shape
        base_error = np.var(y)

        for feature_index in range(n):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_indices = X[:, feature_index] < threshold
                right_indices = ~left_indices

                if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                    continue

                left_variance = np.var(y[left_indices])
                right_variance = np.var(y[right_indices])

                weighted_variance = (np.sum(left_indices) / m) * left_variance + \
                                    (np.sum(right_indices) / m) * right_variance

                gain = base_error - weighted_variance

                if gain > best_gain:
                    best_gain = gain
                    best_feature_index = feature_index
                    best_threshold = threshold

        return best_feature_index, best_threshold, best_gain

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, tree):
        if isinstance(tree, dict):
            feature_index = tree['feature_index']
            threshold = tree['threshold']

            if x[feature_index] < threshold:
                return self._predict_tree(x, tree['left_subtree'])
            else:
                return self._predict_tree(x, tree['right_subtree'])
        else:
            return tree

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.estimators = []

    def fit(self, X, y):
        y_pred = np.full_like(y, np.mean(y))  # initialize predictions with mean of y

        for _ in range(self.n_estimators):
            residuals = y - y_pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.estimators.append(tree)
            y_pred += self.learning_rate * tree.predict(X)

    def predict(self, X):
        y_pred = np.zeros(len(X))
        for estimator in self.estimators:
            y_pred += self.learning_rate * estimator.predict(X)
        return y_pred

def evaluate_performance(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, r2

# Generate synthetic regression data
X, y = make_regression(n_samples=100, n_features=1, noise=0.1, random_state=42)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [1, 2, 3]
}

# Initialize the GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')



#Q4. What is a weak learner in Gradient Boosting?

In [ ]:
"""
In the context of Gradient Boosting, a weak learner refers to a simple or basic model that performs slightly better than random guessing for a given learning task.

"""

#Q5. What is the intuition behind the Gradient Boosting algorithm?

In [ ]:
"""
The intuition behind the Gradient Boosting algorithm revolves around sequentially improving the performance of a model by leveraging the strengths of weak learners and correcting their errors.

"""

#Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?

In [ ]:
"""
The Gradient Boosting algorithm builds an ensemble of weak learners, typically decision trees, in a sequential manner

"""

#Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting algorithm?

In [ ]:
"""
- Loss Function Definition
- Initial Prediction
- Residual Calculation
- Sequential Model Training
- Update Predictions
- Repeat
- Ensemble Combination
- Regularization
- Objective Function Optimization
- Evaluation

"""